In [1]:
import re
import json
import matplotlib.pyplot as plt
import collections
#pip install tabulate
from tabulate import tabulate
import pandas as pd

data_europea = []
with open("europea.json","r") as file:
    data_europea = json.load(file)
    
data_bnf = []
with open("bnf.json","r") as file:
    data_bnf = json.load(file)            

In [ ]:
from flask import render_template
import io
import base64

from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure

@app.route("/mysuperplot", methods=["GET"])
def plotView():

    # Generate plot
    fig = Figure()
    
    axis = fig.add_subplot(1, 2, 1)
    labels = []
    sizes = []
    for x, y in type_dict.items():
        labels.append(x)
        sizes.append(y)
    axis.pie(sizes,labels = labels,autopct = '%1.1f%%')
    axis.set_title('Europeana')
    
    axis2 = fig.add_subplot(1, 2, 2)
    labels = []
    sizes = []
    for x, y in type_dict.items():
        labels.append(x)
        sizes.append(y)
    axis2.pie(sizes,labels = labels,autopct = '%1.1f%%')
    axis2.set_title('Europeana')
    
    # Convert plot to PNG image
    pngImage = io.BytesIO()
    FigureCanvas(fig).print_png(pngImage)
    
    # Encode PNG image to base64 string
    pngImageB64String = "data:image/png;base64,"
    pngImageB64String += base64.b64encode(pngImage.getvalue()).decode('utf8')
    
    return render_template("image.html", image=pngImageB64String)

# In the image.html jinja2 template use the following <img> to add the plot:
# <img src="{{ image }}"/>


In [3]:
year_dict={}
for i in range(0,len(data_europea)):
    if 'year' in data_europea[i]:
        key=data_europea[i]['year'][0]
        if key in year_dict:
            year_dict[key]+=1
        else:
            year_dict[key]=1

sorted_year_dict=collections.OrderedDict(sorted(year_dict.items()))

year_dict2={}
for i in range(0,len(data_bnf)):
    if 'date' in data_bnf[i]:
        date=data_bnf[i]['date'][1]
        #print(date)
        year=re.match(r'.*([1-3][0-9]{3})', date)
        if year is not None:
            #print(year.group(1))
            key=year.group(1)
            if key in year_dict2:
                year_dict2[key]+=1
            else:
                year_dict2[key]=1
sorted_year_dict2=collections.OrderedDict(sorted(year_dict2.items()))

In [ ]:
@app.route("/mysuperplot", methods=["GET"])
def plotView2():

    # Generate plot
    fig = Figure()
    
    axis = fig.add_subplot(1, 1, 1)
    d = {
        'Europeana': sorted_year_dict,
        'BNF': sorted_year_dict2
    }
    axis = pd.DataFrame(d).plot(kind='bar')
    axis.set_title('Results by year')
    axis.set_ylabel("values")
    
    # Convert plot to PNG image
    pngImage = io.BytesIO()
    FigureCanvas(fig).print_png(pngImage)
    
    # Encode PNG image to base64 string
    pngImageB64String = "data:image/png;base64,"
    pngImageB64String += base64.b64encode(pngImage.getvalue()).decode('utf8')
    
    return render_template("image.html", image=pngImageB64String)

# In the image.html jinja2 template use the following <img> to add the plot:
# <img src="{{ image }}"/>
